<a href="https://colab.research.google.com/github/mohamedehab00/A-Hybrid-Arabic-Text-Summarization-Approach-based-on-Transformers/blob/main/MainProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Environment

In [1]:
!pip install transformers==4.8.0 datasets sentencepiece==0.1.96

     |████████████████████████████████| 2.5 MB 5.1 MB/s 
     |████████████████████████████████| 325 kB 34.2 MB/s 
     |████████████████████████████████| 1.2 MB 30.5 MB/s 
     |████████████████████████████████| 3.3 MB 29.8 MB/s 
     |████████████████████████████████| 895 kB 32.5 MB/s 
     |████████████████████████████████| 212 kB 36.2 MB/s 
     |████████████████████████████████| 325 kB 34.2 MB/s 
     |████████████████████████████████| 312 kB 34.8 MB/s 
     |████████████████████████████████| 311 kB 18.7 MB/s 
     |████████████████████████████████| 312 kB 43.3 MB/s 
     |████████████████████████████████| 311 kB 46.0 MB/s 
     |████████████████████████████████| 311 kB 47.5 MB/s 
     |████████████████████████████████| 306 kB 44.3 MB/s 
     |████████████████████████████████| 298 kB 49.2 MB/s 
     |████████████████████████████████| 298 kB 47.7 MB/s 
     |████████████████████████████████| 290 kB 43.5 MB/s 
     |████████████████████████████████| 290 kB 48.6 MB/s 
     |█████████

In [2]:
!pip install nltk

In [3]:
!git clone https://github.com/nadasalah99/bert-extractive-summarizer

Cloning into 'bert-extractive-summarizer'...
remote: Enumerating objects: 276, done.
remote: Total 276 (delta 0), reused 0 (delta 0), pack-reused 276
Receiving objects: 100% (276/276), 57.15 KiB | 1.97 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [4]:
!pip install -r /content/bert-extractive-summarizer/requirements.txt

     |████████████████████████████████| 17.3 MB 561 kB/s 
     |████████████████████████████████| 753.4 MB 7.7 kB/s 
     |████████████████████████████████| 387 kB 45.7 MB/s 
     |████████████████████████████████| 2.1 MB 33.6 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 286 kB 45.4 MB/s 
     |████████████████████████████████| 7.1 MB 34.3 MB/s 
     |████████████████████████████████| 248 kB 40.8 MB/s 
     |████████████████████████████████| 576 kB 43.4 MB/s 
     |████████████████████████████████| 132 kB 44.2 MB/s 
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of neuralcoref to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tqdm to determine which version is compatible with other requirements. This could take

In [5]:
!pip install tqdm

In [6]:
!pip install spacy

# Import libraries

In [7]:
import pandas as pd
import numpy as np
import pickle
import re
import os
import string
import heapq

from transformers import AutoTokenizer, AutoModel, AutoModelForTokenClassification, AutoModelForSeq2SeqLM, pipeline

import torch

import logging

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing,metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
st = ISRIStemmer()
nltk.download('stopwords')
stop=stopwords.words('arabic')

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Text Classification

In [8]:
def clean(text):
  #remove all English chars 
  text = re.sub(r'\s*[A-Za-z]\s*', ' ' , text)
  #remove hashtags
  text = re.sub("#", " ", text)
  #remove all numbers 
  text = re.sub(r'\[0-9]*\]',' ',text)
  #remove duplicated chars
  text = re.sub(r'(.)\1+', r'\1', text)
  #remove :) or :(
  text = text.replace(':)', "")
  text = text.replace(':(', "")
  #remove multiple exclamation
  text = re.sub(r"(\!)\1+", ' ', text)
  #remove multiple question marks
  text = re.sub(r"(\?)\1+", ' ', text)
  #remove multistop
  text = re.sub(r"(\.)\1+", ' ', text)
  #remove additional spaces
  text = re.sub(r"[\s]+", " ", text)
  text = re.sub(r"[\n]+", " ", text)
  
  return text

def remStopWords(Text):
  return " ".join(word for word in Text.split() if word not in stop)

def stemWords(Text):
  return " ".join(st.stem(word) for word in Text.split())
  
def pipeline(Text):
  #preprocessing step
  Text = clean(Text)
  Text = "".join([char for char in Text if char not in string.ascii_letters]).strip()
  Text = remStopWords(str(Text))
  Text = stemWords(Text)
  #vectorize the text
  Text_Vector = tfidf_vect.transform([Text])
  predictions = clf_svm.predict(Text_Vector)
  return encoder.inverse_transform(predictions)[0]

# Extractive Model

In [9]:
cd /content/bert-extractive-summarizer

/content/bert-extractive-summarizer


In [10]:
ls

arabert_summarize_example.py  Makefile                  setup.cfg
Dockerfile                    README.md                 setup.py
Dockerfile.service            requirements-service.txt  summarize.py
example_file.txt              requirements.txt          summarizer/
LICENSE                       server.py                 tests/


In [11]:
from summarizer import Summarizer 

extractive_Model = AutoModel.from_pretrained('asafaya/bert-base-arabic')
extractive_Tokenizer = AutoTokenizer.from_pretrained('asafaya/bert-base-arabic')

extractiveModelSummarizer = Summarizer(custom_model=extractive_Model, custom_tokenizer=extractive_Tokenizer)

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/334k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Model to Pickle

In [12]:
pickle.dump(extractiveModelSummarizer, open("/content/drive/MyDrive/Pickles/extractiveModel.pkl", "wb"))  

# Abstractive Model

In [13]:
model_name = "csebuetnlp/mT5_multilingual_XLSum"
abstractive_Tokenizer = AutoTokenizer.from_pretrained(model_name)
abstractive_Model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [14]:
def abstractiveModelSummarizer(text):
  input_ids = abstractive_Tokenizer(
      [(text)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=1024
  )["input_ids"]

  output_ids = abstractive_Model.generate(
      input_ids=input_ids,
      max_length=600,
      no_repeat_ngram_size=8,
      num_beams=4
  )[0]

  summary = abstractive_Tokenizer.decode(
      output_ids,
      skip_special_tokens=True,
      clean_up_tokenization_spaces=False
  )
  return summary

## Model to Pickle

In [15]:
pickle.dump(abstractiveModelSummarizer, open("/content/drive/MyDrive/Pickles/abstractiveModel.pkl", "wb"))  

# Meta Text Extraction

In [16]:
model_name = "marefa-nlp/marefa-ner"
custom_labels = ["O", "B-job", "I-job", "B-nationality", "B-person", "I-person", "B-location",
                 "B-time", "I-time", "B-event", "I-event", "B-organization", "I-organization",
                 "I-location", "I-nationality", "B-product", "I-product", "B-artwork", "I-artwork"]

NER_tokenizer = AutoTokenizer.from_pretrained(model_name)
NER_Model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(custom_labels))

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [17]:
def extractNER(text: str, model: AutoModelForTokenClassification,tokenizer: AutoTokenizer, start_token: str="▁"):
    
    tokenized_sentence = tokenizer([text], padding=True, truncation=True, return_tensors="pt")
    tokenized_sentences = tokenized_sentence['input_ids'].numpy()

    with torch.no_grad():
        output = model(**tokenized_sentence)

    last_hidden_states = output[0].numpy()
    label_indices = np.argmax(last_hidden_states[0], axis=1)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentences[0])
    special_tags = set(tokenizer.special_tokens_map.values())

    grouped_tokens = []
    for token, label_idx in zip(tokens, label_indices):
        if token not in special_tags:
            if not token.startswith(start_token) and len(token.replace(start_token,"").strip()) > 0:
                grouped_tokens[-1]["token"] += token
            else:
                grouped_tokens.append({"token": token, "label": custom_labels[label_idx]})

    # extract entities
    ents = []
    prev_label = "O"
    for token in grouped_tokens:
        label = token["label"].replace("I-","").replace("B-","")
        if token["label"] != "O":
            
            if label != prev_label:
                ents.append({"token": [token["token"]], "label": label})
            else:
                ents[-1]["token"].append(token["token"])
            
        prev_label = label
    
    # group tokens
    ents = [{"token": "".join(rec["token"]).replace(start_token," ").strip(), "label": rec["label"]}  for rec in ents ]
    
    return ents

In [18]:
def checkedEntities(ents):
    entities = {}
    for element in ents:
      word = wordOnly(element["token"])
      label = element["label"]
      if word not in entities:
        entities[word] = label
    return entities

## Model to Pickle

In [19]:
pickle.dump(NER_tokenizer, open("/content/drive/MyDrive/Pickles/NERTokenizer.pkl", "wb"))  
pickle.dump(NER_Model, open("/content/drive/MyDrive/Pickles/NERModel.pkl", "wb"))  

## KeyWords Extractor

In [20]:
punct = string.punctuation

punct += "،-*)(+=_^%$#@!~^&.?/';.,<>{}"

def wordOnly(word):
  letters = list(word)
  for i in range(len(letters)):
    if letters[i] in punct:
      letters[i] = ""
  return "".join(letters)

In [21]:
def keywords(ents):
  last_dic={}
  for element in ents:
    for key,value in element.items():
        if key=='token':
          value=wordOnly(value)
          if value in last_dic:
            last_dic[value] +=1
          else:
            last_dic[value]=1  
  maxWord = int(len(last_dic) * 0.5)
  
  if maxWord > 5:
    maxWord = 5
  return heapq.nlargest(maxWord, last_dic, key=last_dic.get)  

# Load Pickles

In [22]:
# for reading also binary mode is important
clf_svm = pickle.load(open('/content/drive/MyDrive/Pickles/TextClassifier.pkl', 'rb'))
  
encoder = pickle.load(open('/content/drive/MyDrive/Pickles/LabelEncoder.pkl', 'rb'))

tfidf_vect = pickle.load(open('/content/drive/MyDrive/Pickles/TextVectorizer.pkl', 'rb'))

extractiveModel = pickle.load(open('/content/drive/MyDrive/Pickles/extractiveModel.pkl', 'rb'))

abstractiveModel = pickle.load(open('/content/drive/MyDrive/Pickles/abstractiveModel.pkl', 'rb'))

NER_tokenizer = pickle.load(open("/content/drive/MyDrive/Pickles/NERTokenizer.pkl", 'rb'))

NER_Model = pickle.load(open("/content/drive/MyDrive/Pickles/NERModel.pkl", 'rb'))

# The Models Output

In [23]:
textInput = """تحت عنوان "أزياء مشعة" قدم المصمم اللبناني بسيل سودة هذا الشهر عرضه في روما،
وهو العرض السابع بعد عدة عروض ناجحة أقامها في بيروت، انتقد سودة هجرة المصممين
العرب إلى الدول الأوروبية لتقديم عروضهم، وطالب بتأسيس أسبوع عربي للموضة
بمواصفات عالمية، يقدم الإبداعات العربية ويعمل على تطوير المهنة، ونفى احتكار
الرجل التصميم للمرأة، مؤكدا أن قدرة المرأة على التصميم للرجل، وآراء أخرى في
هذا الحوار.
 * كيف كان عرضك الأخير في روما وما المجموعة التي قدمتها؟
 ـ قدمت في عرض روما مجموعة من 40 قطعة لموسم الصيف مستوحاة من فترة السبعينيات،
استخدمت فيها الأقمشة التي تستخدم عادة في أزياء الهوت كوتيور مثل الموسلين
والحرير، ولكن بطريقة جديدة مع استخدام الإكسسوار، العرض أقيم تحت عنوان "أزياء
مشعة"، وعلى المستوى العربي كانت الأصداء جيدة جدا.
 * ما هي الألوان والأقمشة التي تفضل استعمالها في تصاميمك؟
 ـ أفضل عادة ألوان الباستيل من الأصفر إلى البيتش سومو والتركواز والبيج والزهر،
وهذه هي ألوان الموضة، وهناك الأسود الذي مازال محافظا على رونقه، والأبيض كذلك،
أما القماش فيحدد حسب موضة العام، فالموضة تحتم عليك نوعية قماش محددة وتضعك في
إطار معين.
 * ما هي المعوقات لمجال تصميم الأزياء في لبنان؟
 ـ المشكلة في عدم وجود أسبوع سنوي للموضة يقام في لبنان على نطاق عالمي، وهجرة
العقول المبدعة، فالمصممون يتركون بلدهم ويعرضون في الخارج، والمفروض أن نقيم
أسبوعا للموضة في العالم العربي بشكل عالمي، فيحدث من خلاله النقد والتطوير، كما
يحدث في روما ولندن ونيويورك اليوم، يجب أن نعمل اليوم قبل الغد على تأسيس هذا
الأسبوع، ولبنان أكثر بلد عنده مقومات إقامة أسبوع موضة، فلديه الأيدي العاملة
والتقنية، مما يمكنه من إقامة أسبوع للموضة، يدرج على المستوى العالمي.
 * لماذا ترتبط الموضة غالبا بارتفاع السعر، وكيف يمكن تحقيق معادلة الأناقة
للجميع؟
 ـ أكيد إن الأناقة حق للجميع، ولكن الهوت كوتيور لا يمكن تقديمه للجميع، تماما
مثل الشخص الذي يشتري منزلا غاليا ويعد له الديكور الخاص به، ولكن عن طريق
الفستان الجاهز والخياطين يمكن أن نقدم تصميمات أنيقة للجميع، ولكن الهوت كوتيور
لا يمكن أن يكون للجميع، لأنها نوعية من الأزياء يتم ارتداؤها في مناسبات معينة
خاصة جدا، مثل الأفراح والحفلات والمناسبات السعيدة، فله ميزات خاصة مكلفة، ويجب
أن نحافظ على هذه الميزات.
 * لماذا يحتكر الرجل مهنة التصميم للمرأة؟
 ـ في العالم توجد مصممات كثيرات نجحن في التصميم للمرأة، ولكن في العالم العربي
يندر ذلك لظروف اجتماعية مختلفة، أيضا المرأة لو صممت تصميما جيدا بفطرتها ستقول
هذا التصميم لي، ليس في الأمر احتكار، بالعكس نحن لا نرفض الفكرة فالمرأة قادرة
على الإبداع، والرجولة لا تحتكر تصميم الأزياء.
 * صممت إحدى الشركات اليابانية روبوت خاصاً لعرض الأزياء، فهل تنجح هذه الفكرة؟.
 ـ الموديل يجب أن تعرضه امرأة، ولا أتصور أن يتم بخلاف ذلك، فلا يمكن أن أعرض
تصميماتي على روبوت، هذه تكنولوجيا ولا أتصور تطبيقها في الهوت كوتيور، هناك
محاولات تمت من هذا القبيل، مثال عرض سينمائي أقيم في التسعينيات نفذه مصمم فرنسي
اسمه هيري موجلير، ونجح العرض ولكن كان مكلفا، فقد كلف مليون دولار، وكانت محاولة
في فترة تحول للعروض السينمائية، وهناك محاولات أخرى سابقة تمت، ولكنها كانت
بمثابة رسائل وتنبيه إلى أن الإنسان تحول إلى روبوت، وأننا ذاهبون إلى الميكنة،
ولكن لا يمكن أن نستعيض عن المرأة أو الرجل بروبوت.
 * كيف تأتي فكرة التصميم؟
 ـ أحيانا يكون كالإلهام فقد يأتي فجأة، وقد يكون نتيجة بحث، بعد أن تضع نفسك في
جو معين من الموسيقي والسينما والنحت والرسم، فهو إنتاج فني متكامل، تظهر معه
أشياء من الجو الذي تعيش فيه، أو تحب أن تكون فيه، إنها حالة كالحلم، ترسم موديل
ثم تنفذه أو تنفذه مباشرة، وذلك حسب الفكرة وكيفية تطورها.
 * ما رأيك ببعض المصممين الذين تمردوا على السمات الأساسية لفستان الزفاف اللون
الأبيض؟ 
ـ تطوير فستان الفرح ممكن، ولكن اللون الأبيض أساسي، فلا يمكن أن نصممه أسود أو
أحمر، يمكن أن يكون لونه رمادياً فاتحاً أو أبيض أو ذهبياً، ولكن عدا ذلك لا
أعتقد، ومن الصعب وجود مصمم يتخطى هذه الحدود، وإذا وجدت هذه الألوان في العروض
فإن 99% من العرائس لا يرتدونه، ولكن هذا لا يمنع أن يكون جميلا.
 * ما دور الإكسسوارات في تصاميمك؟
 ـ ركزت على الإكسسوارات في بعض موديلات هذه المجموعة، وفي المجموعة السابقة قدمت
فساتين مصنوعة من الإكسسوار، ولكن بطريقتي الخاصة، وكانت مستوحاة من الجو الماسي،
وأنا أفضل مجوهرات الكريستال، وأن يكون اللوك العام للإكسسوارات مجوهرات.
 * كيف يمكن الارتقاء بمهنة التصميم؟
 ـ على الدول العربية تشجيع المهنة لكي نتطور ونصل إلى المستوى العالمي، كل دور
الأزياء مرتبطة لدينا بأشخاص، في الخارج دور الأزياء كيانات كبيرة، المصمم ليس
صاحبها، وتوجد فيها هيكلية في الإدارة، وهذا قليل في العالم العربي، وإذا وجد
لدينا سيساعد المصمم كثيرا على التصميم، بعيدا عن التسويق والإدارة، ولم نصل بعد
إلى هذه المرحلة.
 في الوطن العربي هناك معدل إنفاق عال على الأزياء، ولكن لا يوجد تطور، ومهنة
الهوت كوتيور صناعة آخذة في التراجع، وفي أوروبا أصبحوا يستخدمونها في الإعلام
وفي تسويق أسمائهم، لا يهمهم أن يبيعوا، ومن خلالها يتم تسويق مواد أخرى تحت هذا
الاسم، فهم ينفقون ملايين الدولارات على العروض، ولكن تحت إطار تسويقي وإعلامي،
وعندنا يتهم المصممون العرب بأن ليس لديهم إبداع، إما أن تبدع أو تبيع."""

In [24]:
the_result_extractive = extractiveModel(textInput)
print(the_result_extractive)

تحت عنوان "أزياء مشعة" قدم المصمم اللبناني بسيل سودة هذا الشهر عرضه في روما،
وهو العرض السابع بعد عدة عروض ناجحة أقامها في بيروت، انتقد سودة هجرة المصممين
العرب إلى الدول الأوروبية لتقديم عروضهم، وطالب بتأسيس أسبوع عربي للموضة
بمواصفات عالمية، يقدم الإبداعات العربية ويعمل على تطوير المهنة، ونفى احتكار
الرجل التصميم للمرأة، مؤكدا أن قدرة المرأة على التصميم للرجل، وآراء أخرى في
هذا الحوار. * ما هي المعوقات لمجال تصميم الأزياء في لبنان؟ ـ الموديل يجب أن تعرضه امرأة، ولا أتصور أن يتم بخلاف ذلك، فلا يمكن أن أعرض
تصميماتي على روبوت، هذه تكنولوجيا ولا أتصور تطبيقها في الهوت كوتيور، هناك
محاولات تمت من هذا القبيل، مثال عرض سينمائي أقيم في التسعينيات نفذه مصمم فرنسي
اسمه هيري موجلير، ونجح العرض ولكن كان مكلفا، فقد كلف مليون دولار، وكانت محاولة
في فترة تحول للعروض السينمائية، وهناك محاولات أخرى سابقة تمت، ولكنها كانت
بمثابة رسائل وتنبيه إلى أن الإنسان تحول إلى روبوت، وأننا ذاهبون إلى الميكنة،
ولكن لا يمكن أن نستعيض عن المرأة أو الرجل بروبوت. ـ ركزت على الإكسسوارات في بعض موديلات هذه المجموعة، وف

In [26]:
 the_result_abstractive = abstractiveModel(textInput)  
print(the_result_abstractive)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


عرضت مصممة الأزياء اللبنانية بسيل سودة هذا الشهر في روما، وهو العرض السابع بعد عدة عروض ناجحة أقامها في بيروت.


In [27]:
the_result_hyrbid = abstractiveModel(the_result_extractive)
print(the_result_hyrbid)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


عرض مصمم الأزياء اللبناني بسيل سودة هذا الشهر عرضه في روما، وهو العرض السابع بعد عدة عروض ناجحة أقامها في بيروت، وطالب بتأسيس أسبوع عربي للموضة بمواصفات عالمية.


In [28]:
textClass = pipeline(textInput).replace("'","")
print(textClass)

القانون-علوم اجتماعية


In [31]:
ents = extractNER(text=textInput, model=NER_Model, tokenizer=NER_tokenizer, start_token="▁")

In [32]:
keyWords = keywords(ents)
print(keyWords)

['روما', 'الهوت كوتيور', 'المصمم', 'اللبناني', 'بسيل سودة']


In [33]:
ents = checkedEntities(ents)
print(ents)

{'المصمم': 'job', 'اللبناني': 'nationality', 'بسيل سودة': 'person', 'روما': 'location', 'بيروت': 'location', 'سودة': 'person', 'المصممين': 'job', 'الدول الأوروبية': 'location', 'السبعينيات': 'time', 'الهوت كوتيور': 'artwork', 'لبنان؟': 'location', 'لبنان': 'location', 'العالم العربي': 'location', 'روما ولندن ونيويورك': 'location'}
